In [53]:
from sympy import *
init_printing()

In [54]:
def is_zero(A):
    return A == zeros(A.shape[0], A.shape[1])

def leftmult2(m, i0, i1, a, b, c, d):
    for j in range(m.cols):
        x, y = m[i0, j], m[i1, j]
        m[i0, j] = a * x + b * y
        m[i1, j] = c * x + d * y

def rightmult2(m, j0, j1, a, b, c, d):
    for i in range(m.rows):
        x, y = m[i, j0], m[i, j1]
        m[i, j0] = a * x + c * y
        m[i, j1] = b * x + d * y

def smith(m, domain=ZZ):
    m = Matrix(m)
    s = eye(m.rows)
    t = eye(m.cols)
    last_j = -1
    for i in range(m.rows):
        for j in range(last_j+1, m.cols):
            if not is_zero(m.col(j)):
                break
        else:
            break
        if m[i,j] == 0:
            for ii in range(m.rows):
                if m[ii,j] != 0:
                    break
            leftmult2(m, i, ii, 0, 1, 1, 0)
            rightmult2(s, i, ii, 0, 1, 1, 0)
        rightmult2(m, j, i, 0, 1, 1, 0)
        leftmult2(t, j, i, 0, 1, 1, 0)
        j = i
        upd = True
        while upd:
            upd = False
            for ii in range(i+1, m.rows):
                if m[ii, j] == 0:
                    continue
                upd = True
                if domain.rem(m[ii, j], m[i, j]) != 0:
                    coef1, coef2, g = domain.gcdex(m[i,j], m[ii, j])
                    coef3 = domain.quo(m[ii, j], g)
                    coef4 = domain.quo(m[i, j], g)
                    leftmult2(m, i, ii, coef1, coef2, -coef3, coef4)
                    rightmult2(s, i, ii, coef4, -coef2, coef3, coef1)
                coef5 = domain.quo(m[ii, j], m[i, j])
                leftmult2(m, i, ii, 1, 0, -coef5, 1)
                rightmult2(s, i, ii, 1, 0, coef5, 1)
            for jj in range(j+1, m.cols):
                if m[i, jj] == 0:
                    continue
                upd = True
                if domain.rem(m[i, jj], m[i, j]) != 0:
                    coef1, coef2, g = domain.gcdex(m[i,j], m[i, jj])
                    coef3 = domain.quo(m[i, jj], g)
                    coef4 = domain.quo(m[i, j], g)
                    rightmult2(m, j, jj, coef1, -coef3, coef2, coef4)
                    leftmult2(t, j, jj, coef4, coef3, -coef2, coef1)
                coef5 = domain.quo(m[i, jj], m[i, j])
                rightmult2(m, j, jj, 1, -coef5, 0, 1)
                leftmult2(t, j, jj, 1, coef5, 0, 1)
        last_j = j
    for i1 in range(min(m.rows, m.cols)):
        for i0 in reversed(range(i1)):
            coef1, coef2, g = domain.gcdex(m[i0, i0], m[i1,i1])
            if g == 0:
                continue
            coef3 = domain.quo(m[i1, i1], g)
            coef4 = domain.quo(m[i0, i0], g)
            leftmult2(m, i0, i1, 1, coef2, coef3, coef2*coef3-1)
            rightmult2(s, i0, i1, 1-coef2*coef3, coef2, coef3, -1)
            rightmult2(m, i0, i1, coef1, 1-coef1*coef4, 1, -coef4)
            leftmult2(t, i0, i1, coef4, 1-coef1*coef4, 1, -coef1)
    return (s, m, t)

In [55]:
def P_op(In_size, n, m):
    return eye(In_size).elementary_row_op('n<->m', n, m)

def M_op(In_size, n, k):
    return eye(In_size).elementary_row_op('n->kn', n, k)

def A_op(In_size, n, m, k):
    return eye(In_size).elementary_row_op('n->n+km', row1=n, row2=m, k=k)

def smart_rref_decomposition(rref):
    elementary_matrices = []
    # количество нулевых строк в rref
    null_row_count = 0
    row = 0
    # вычеркиваем нулевые строки
    while row < rref.shape[0]:
        if rref.row(row) == zeros(1, rref.shape[1]):
            rref.row_del(row)
        row += 1
    # индексы ячеек главной диагонали
    diag_i, diag_j = 0, 0
    while diag_i < rref.shape[0] and diag_j < rref.shape[1]:
        element_to_one = rref[diag_i, diag_j]
        if element_to_one == 0:
            # ищем ненулевой элемент в столбце
            found = False
            for row in range(diag_i + 1, rref.shape[0]):
                # если нашли
                if rref[row, diag_j] != 0:
                    element_to_one = rref[row, diag_j]
                    # перестановка
                    op = P_op(rref.shape[0], diag_i, row)
                    # сохраняем перестановку
                    elementary_matrices.append(op)
                    rref = op * rref
                    found = True
                    break
            # если не нашли
            if found == False:
                # смещаемся вправо
                diag_j += 1
                continue  
        # приводим к единице, если надо
        if element_to_one != 1: 
            op = M_op(rref.shape[0], diag_i, 1 / element_to_one)
            elementary_matrices.append(op)
            rref = op * rref   
        # приводим к нулю элементы под и над данной ячейкой
        row = 0 
        while row < rref.shape[0]:
            if row != diag_i and rref[row, diag_j] != 0 and rref.row(row) != zeros(1, rref.shape[1]):
                element_to_zero = rref[row, diag_j]
                op = A_op(rref.shape[0], row, diag_i, -element_to_zero)
                elementary_matrices.append(op)
                rref = op * rref  
                # если получили нулевую строку,
                # то вычеркиваем ее
                if rref.row(row) == zeros(1, rref.shape[1]):
                    rref.row_del(row)  
                    row -= 1
                    if (row < diag_i):
                        diag_i -= 1
                    # и запоминаем, что вычеркнули
                    null_row_count += 1
            row += 1         
        diag_i += 1
        diag_j += 1            
    # записываем обратно вычеркнутые строки
    for i in range(null_row_count):
        rref = rref.row_insert(rref.shape[0], zeros(1, rref.shape[1]))
    return rref, elementary_matrices

In [56]:
# разложение невырожденных матриц на произведение элементарных
def nondegenerate_decomposition(A):
    rref, elementary_matrices = smart_rref_decomposition(A)
    elementary_matrices.append(rref)
    elementary_matrices_reversed = list(reversed(elementary_matrices))
    elementary_matrices_reversed_inv = [i.inv() for i in elementary_matrices_reversed]
    return elementary_matrices_reversed_inv

# разложение с дробными коэффициентами
def smart_snf_decomposition(A):
    m, n = A.shape
    A_Im = Matrix.hstack(A, eye(m))
    U = A_Im.rref()[0][:, n:]
    R = A.rref()[0]
    RT_In = Matrix.hstack(R.T, eye(n))
    VT = RT_In.rref()[0][:, m:]
    V = VT.T
    return U*A*V, nondegenerate_decomposition(U), nondegenerate_decomposition(V)

# разложение с целыми коэффициентами
def smart_snf_decomposition_over_ring_zz(A):
    S, SNF, T = smith(A)
    # S*SNF*T == A
    U = S.inv()
    V = T.inv()
    #U*A*V == SNF
    return U*A*V, nondegenerate_decomposition(U), nondegenerate_decomposition(V)

In [57]:
#A = Matrix([
#    [1,-1,2,1],
#    [2,-1,0,3],
#    [0,1,-4,1]
#])
#A = Matrix([
#    [-6, 111, -36, 6],
#    [5, -672, 210, 74],
#    [0, -255, 81, 24],
#    [-7, 255, -81, -10]
#])
A = Matrix([
    [1,-1,1,2],
    [2,-2,1,-1],
    [-1,1,0,3]
])
SNF, U_decomposition, V_decomposition = smart_snf_decomposition_over_ring_zz(A)
SNF, U_decomposition, V_decomposition

⎛                                                                             
⎜⎡1  0  0  0⎤  ⎡⎡1  0  0⎤  ⎡1   0   0⎤  ⎡1  -1/3  0⎤  ⎡1   0    0⎤  ⎡1   0  0⎤
⎜⎢          ⎥  ⎢⎢       ⎥  ⎢         ⎥  ⎢          ⎥  ⎢          ⎥  ⎢        ⎥
⎜⎢0  1  0  0⎥, ⎢⎢0  1  0⎥, ⎢0   1   0⎥, ⎢0   1    0⎥, ⎢0  -1/3  0⎥, ⎢0   1  0⎥
⎜⎢          ⎥  ⎢⎢       ⎥  ⎢         ⎥  ⎢          ⎥  ⎢          ⎥  ⎢        ⎥
⎜⎣0  0  0  0⎦  ⎣⎣0  0  1⎦  ⎣0  2/3  1⎦  ⎣0   0    1⎦  ⎣0   0    1⎦  ⎣-1  0  1⎦
⎝                                                                             

                          ⎡⎡1  0  0  0⎤  ⎡1  0  0  0 ⎤  ⎡1  0  0  0⎤  ⎡1  0  0
  ⎡1   0  0⎤  ⎡3  0  0⎤⎤  ⎢⎢          ⎥  ⎢           ⎥  ⎢          ⎥  ⎢       
  ⎢        ⎥  ⎢       ⎥⎥  ⎢⎢0  1  0  0⎥  ⎢0  1  0  0 ⎥  ⎢0  1  0  2⎥  ⎢0  1  0
, ⎢-1  1  0⎥, ⎢0  1  0⎥⎥, ⎢⎢          ⎥, ⎢           ⎥, ⎢          ⎥, ⎢       
  ⎢        ⎥  ⎢       ⎥⎥  ⎢⎢0  0  1  0⎥  ⎢0  0  1  -5⎥  ⎢0  0  1  0⎥  ⎢0  0  1
  ⎣0   0  1⎦  ⎣0  0  1⎦⎦  ⎢⎢          ⎥  ⎢         

In [58]:
U = 1
for e_inv in U_decomposition:
    U = e_inv * U
V = 1
for e_inv in V_decomposition:
    V = e_inv * V
U, V, SNF, U*A*V

⎛             ⎡0  -1  -1  3 ⎤                            ⎞
⎜⎡3   -1  0⎤  ⎢             ⎥  ⎡1  0  0  0⎤  ⎡1  0  0  0⎤⎟
⎜⎢         ⎥  ⎢1  1   -1  0 ⎥  ⎢          ⎥  ⎢          ⎥⎟
⎜⎢-1  0   0⎥, ⎢             ⎥, ⎢0  1  0  0⎥, ⎢0  1  0  0⎥⎟
⎜⎢         ⎥  ⎢1  1   0   -5⎥  ⎢          ⎥  ⎢          ⎥⎟
⎜⎣-1  1   1⎦  ⎢             ⎥  ⎣0  0  0  0⎦  ⎣0  0  0  0⎦⎟
⎝             ⎣0  0   0   1 ⎦                            ⎠

In [59]:
S, SNF, T = smith(A)
S*SNF*T == A

True

In [60]:
U = S.inv()
V = T.inv()
U*A*V == SNF

True